In [1]:
# Set API Key from https://sec-api.io/profile

API_KEY = '0b58875db6907cd6f148390143e8aac4bcf04f84960f0721bd0c8b13b2713bf2' #from my github one, all out

In [2]:
# Define get_10k function to return list of filings

from sec_api import QueryApi
from sec_api import ExtractorApi


def get_10_k(ticker):
  list_of_texts=[]
  queryApi = QueryApi(api_key=API_KEY)
  query = {
    "query": { "query_string": {
        "query": "ticker:" +ticker+" AND filedAt:{2013-01-01 TO 2023-12-31} AND formType:\"10-K\""
      } },
    "from": "0",
    "size": "10",
    "sort": [{ "filedAt": { "order": "desc" } }]
  }
  filings = queryApi.get_filings(query)['filings']
  return filings

In [112]:
import pandas as pd

companies = pd.read_excel('/Users/andrewscott/Documents/USF/Spring24/text_analytics/final project/companieslist.xlsx',names=['company'])

In [113]:
# Extract Ticker
companies['ticker'] = companies['company'].apply(lambda x: x[x.find("(")+1:x.find(")")])


In [182]:
companies.head()

,company,ticker
0,"Advanced Micro Devices, Inc. (AMD)",AMD
1,"Airbnb, Inc. (ABNB)",ABNB
2,Alphabet Inc. (GOOGL),GOOGL
3,Amazon.com Inc. (AMZN),AMZN
4,"ANSYS, Inc. (ANSS)",ANSS


In [115]:
import csv

# run the filings query and save the filings data to csv
# will run out of API calls here

for ticker in companies['ticker']:
    filings = get_10_k(ticker)
    field_names = ["id", "accessionNo", "companyName", "companyNameLong", "ticker", 
               "cik", "filedAt", "items", "formType", "periodOfReport", 
               "linkToHtml", "linkToFilingDetails", "linkToTxt", "description", 
               "documentFormatFiles", "dataFiles", "seriesAndClassesContractsInformation",  
               "linkToXbrl", "entities","effectivenessDate"]
 
    with open('filings.csv', 'a', encoding='UTF8') as f:
       writer = csv.DictWriter(f, fieldnames=field_names)
       writer.writeheader()
       writer.writerows(filings)

Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}

In [164]:
# Bring that CSV back into a df

df = pd.read_csv('filings.csv')
links = df['linkToHtml'].tolist()
links = links[:100]
len(links)

100

In [21]:
META = get_10_k('META')

In [27]:
import pandas as pd

# new api key
API_KEY = '0b58875db6907cd6f148390143e8aac4bcf04f84960f0721bd0c8b13b2713bf2'
extractorApi = ExtractorApi(API_KEY)

def get_the_10k(ticker):
    co = get_10_k(ticker)

    df = pd.DataFrame(co)

    list_of_texts = []
    for i,link in enumerate(df['linkToHtml']):
        item_1_a_text  = extractorApi.get_section(link, '1A', 'text')
        list_of_texts.append(item_1_a_text)
    df['texts'] = list_of_texts

    df.to_pickle(f'10kData_{ticker}.pkl')


In [33]:
get_the_10k('NVDA')

In [23]:
# new api key
API_KEY = '0b58875db6907cd6f148390143e8aac4bcf04f84960f0721bd0c8b13b2713bf2'
extractorApi = ExtractorApi(API_KEY)

#run extractor for every link in catalog up to 100
list_of_texts = []
for i,link in enumerate(META_DF['linkToHtml']):
    item_1_a_text  = extractorApi.get_section(link, '1A', 'text')
    list_of_texts.append(item_1_a_text)
META_DF['texts'] = list_of_texts

In [22]:
import pandas as pd
META_DF = pd.DataFrame(META)
META_DF

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
0,ba8f4eee713c44cbd8d6678cc8cf9fab,0001326801-23-000013,1326801,META,"Meta Platforms, Inc.","Meta Platforms, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-01T20:26:31-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'companyName': 'Meta Platforms, Inc. (Filer)...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '14', 'description': 'XBRL TAXON...",[],2022-12-31,NaN
1,3820f32b4f59a0c86e41237afd974d06,0001326801-22-000018,1326801,META,"Meta Platforms, Inc.","Meta Platforms, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-02T21:11:04-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2583141', 'documen...","[{'sequence': '12', 'size': '53905', 'document...",[],2021-12-31,NaN
2,79fc76fad5d30b8a8250a27793aeb165,0001326801-21-000014,1326801,META,Facebook Inc,Facebook Inc (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-01-27T21:13:48-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2768590', 'documen...","[{'sequence': '9', 'size': '52810', 'documentU...",[],2020-12-31,2020-12-31
3,306095d5309b102e4f2c3c8679dd3136,0001326801-20-000013,1326801,META,Facebook Inc,Facebook Inc (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-01-29T21:12:14-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '3015222', 'documen...","[{'sequence': '9', 'size': '54693', 'documentU...",[],2019-12-31,NaN
4,11af44721220f4c23ec457ae916737e1,0001326801-19-000009,1326801,META,Facebook Inc,Facebook Inc (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2019-01-31T16:58:29-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2319378', 'documen...","[{'sequence': '9', 'size': '1878872', 'documen...",[],2018-12-31,NaN
5,54a743da2445661a208c1d150054b509,0001326801-18-000009,1326801,META,Facebook Inc,Facebook Inc (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2018-02-01T16:58:26-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2325088', 'documen...","[{'sequence': '9', 'size': '1948840', 'documen...",[],2017-12-31,NaN
6,3f42f0ea17cf202fa9509edb3b35b957,0001326801-17-000007,1326801,META,Facebook Inc,Facebook Inc (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2017-02-02T21:13:52-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2452404', 'documen...","[{'sequence': '9', 'size': '2632692', 'documen...",[],2016-12-31,NaN
7,1b184a9b72abe709aeb26e3eff81877e,0001326801-16-000063,1326801,META,Facebook Inc,Facebook Inc (Filer),10-K/A,Form 10-K/A - Annual report [Sectio

In [24]:
import pickle

META_DF.to_pickle('10kData_META.pkl')

In [183]:
len(list_of_texts)

85

In [172]:
# Convert list of text items to series to be concated with df, columns dont align run out of calls

texts = pd.Series(list_of_texts,name='texts')
df = pd.concat([df, texts], axis=1)


In [2]:
import pickle
import pandas as pd

df.to_pickle('10kData.pkl')

NameError: name 'df' is not defined

# @ Swathi start here

In [10]:
import pandas as pd
new_df = pd.read_pickle('10kData.pkl')

# Print the tickers we have in the df
print('tickers: ',new_df['ticker'].unique())

# Print how many filing URLs havent been pulled 1a section yet
print('count: ',new_df['texts'].isnull().count())

display(new_df.head())

tickers:  ['AMD' 'ABNB' 'GOOGL' 'AMZN' 'ANSS' 'AAPL' 'AMAT' 'TEAM' 'AVGO' 'CDNS'
 'CIEN' 'CSCO' 'CTXS' 'NET' 'CTSH']
count:  119


,id,accessionNo,companyName,companyNameLong,ticker,cik,filedAt,items,formType,periodOfReport,...,linkToFilingDetails,linkToTxt,description,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,linkToXbrl,entities,effectivenessDate,texts
0,adfd38b73247c5c3d782aa318ebfbc1b,0000002488-23-000047,ADVANCED MICRO DEVICES INC,ADVANCED MICRO DEVICES INC (Filer),AMD,2488,2023-02-27T16:20:39-05:00,NaN,10-K,12/31/22,...,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '10', 'description': 'XBRL TAXON...",[],NaN,[{'companyName': 'ADVANCED MICRO DEVICES INC (...,NaN,ITEM 1A. RISK FACTORS \n\nThe risks and uncer...
1,f7720015403fbe0f70560cbea0729fbf,0000002488-22-000016,ADVANCED MICRO DEVICES INC,ADVANCED MICRO DEVICES INC (Filer),AMD,2488,2022-02-03T17:22:50-05:00,NaN,10-K,12/25/21,...,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '10', 'description': 'XBRL TAXON...",[],NaN,[{'companyName': 'ADVANCED MICRO DEVICES INC (...,NaN,ITEM 1A. RISK FACTORS \n\nThe risks and uncer...
2,e5f089f6bd207ce3e9374110e4c7f199,0001628280-21-001185,ADVANCED MICRO DEVICES INC,ADVANCED MICRO DEVICES INC (Filer),AMD,2488,2021-01-29T17:24:00-05:00,NaN,10-K,12/26/20,...,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'size': '2403831', 'documen...","[{'sequence': '11', 'size': '58503', 'document...",[],NaN,"[{'fiscalYearEnd': '1226', 'stateOfIncorporati...",12/26/20,ITEM 1A. RISK FACTORS \n\nThe risks and uncer...
3,a5ee01505dde1dcae61178519fe56ab3,0000002488-20-000008,ADVANCED MICRO DEVICES INC,ADVANCED MICRO DEVICES INC (Filer),AMD,2488,2020-02-04T17:21:47-05:00,NaN,10-K,12/28/19,...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/2488/0...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '10', 'description': 'XBRL TAXON...",[],NaN,[{'companyName': 'ADVANCED MICRO DEVICES INC (...,NaN,ITEM 1A. \n\nRISK FACTORS \n\n##TABLE_END\n\n...
4,001dc2d78045eddf8fbeb1583ed529ae,0000002488-19-000011,ADVANCED MICRO DEVICES INC,ADVANCED MICRO DEVICES INC (Filer),AMD,2488,2019-02-08T16:44:26-05:00,NaN,10-K,12/29/18,...,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '12', 'description': 'XBRL INSTA...",[],NaN,[{'companyName': 'ADVANCED MICRO DEVICES INC (...,NaN,ITEM 1A. RISK FACTORS \n\nThe risks and uncer...


In [12]:
# If this returns >0 then there is a bad url and the code will break
print('Bad Urls',len(new_df[~new_df['linkToTxt'].str.contains('http')]['linkToTxt']))

Bad Urls 0


In [29]:
## Prepare the links we are going to fetch text sections for
# in this case, its everything that is null in the df
links = new_df.loc[new_df['texts'].isnull(),]
links.head(2)


,id,accessionNo,companyName,companyNameLong,ticker,cik,filedAt,items,formType,periodOfReport,...,linkToFilingDetails,linkToTxt,description,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,linkToXbrl,entities,effectivenessDate,texts
86,459e0b0e60215169e9e39274bfb6be9b,0000858877-23-000023,"CISCO SYSTEMS, INC.","CISCO SYSTEMS, INC. (Filer)",CSCO,858877,2023-09-07T16:30:09-04:00,NaN,10-K,7/29/23,...,https://www.sec.gov/Archives/edgar/data/858877...,https://www.sec.gov/Archives/edgar/data/858877...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '9', 'description': 'XBRL TAXONO...",[],NaN,"[{'companyName': 'CISCO SYSTEMS, INC. (Filer)'...",NaN,NaN
87,3eaa4f30cfec02a2a09c9a43c9c89654,0000858877-22-000013,"CISCO SYSTEMS, INC.","CISCO SYSTEMS, INC. (Filer)",CSCO,858877,2022-09-08T16:47:54-04:00,NaN,10-K,7/30/22,...,https://www.sec.gov/Archives/edgar/data/858877...,https://www.sec.gov/Archives/edgar/data/858877...,Form 10-K - Annual report [Section 13 and 15(d...,"[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],NaN,"[{'companyName': 'CISCO SYSTEMS, INC. (Filer)'...",NaN,NaN


In [26]:
from sec_api import ExtractorApi

API_KEY = '0abcd7241f9ce6276e85b83e934d6fbce4951b381765cfbe0e8d547217b12ac6'
extractorApi = ExtractorApi(API_KEY)

#run extractor for every link in catalog up to 100
for i,link in enumerate(links['linkToHtml']):
    item_1_a_text  = extractorApi.get_section(link, '1A', 'text')
    link = item_1_a_text

Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}

In [30]:
# merge back into df
new_df.update(links)



## Below is cleaning and code, above is getting the data

In [43]:
import re


def clean_risk(doc):
    doc_cleaned = re.sub(r'&#\d+;', '', doc)
    return(doc_cleaned)

AAPL_split = clean_risk(AAPL[0]).split('\n')
[doc for doc in AAPL_split if len(doc) > 40]

['The Companys business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Companys business, reputation, results of operations, financial condition and stock price can be materially and adversely affected. ',
 'Because of the following factors, as well as other factors affecting the Companys results of operations and financial condition, past financial performance should not be considered to be a reliable indicator of future performance, and investors should not use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains forward-looking statements. ',
 'This section should be read in conjunction with Part II, Item 7, Managements Discussion and Analysis of Financial Condition and Results of Operations and the consolidated financial sta

In [24]:
#print(MSFT[0])
MSFT[0].split('\n')

[' ITEM 1A. RIS K FACTORS ',
 '',
 'Our operations and financial results are subject to various risks and uncertainties, including those described below, that could adversely affect our business, financial condition, results of operations, cash flows, and the trading price of our common stock. ',
 '',
 'STRATEGIC AND COMPETITIVE RISKS ',
 '',
 'We face intense competition across all markets for our products and services, which may lead to lower revenue or operating margins. ',
 '',
 'Competition in the technology sector ',
 '',
 'Our competitors range in size from diversified global companies with significant research and development resources to small, specialized firms whose narrower product lines may let them be more effective in deploying technical, marketing, and financial resources. Barriers to entry in many of our businesses are low and many of the areas in which we compete evolve rapidly with changing and disruptive technologies, shifting user needs, and frequent introductions 

In [69]:
from bs4 import BeautifulSoup
import requests

URL = 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm'

response = requests.get(URL)
print(response.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<title>SEC.gov | Request Rate Threshold Exceeded</title>
<style>
html {height: 100%}
body {height: 100%; margin:0; padding:0;}
#header {background-color:#003968; color:#fff; padding:15px 20px 10px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px; border-bottom:solid 5px #000;}
#footer {background-color:#003968; color:#fff; padding:15px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px;}
#content {max-width:650px;margin:60px auto; padding:0 20px 100px 20px; background-image:url(seal_bw.png);background-repeat:no-repeat;background-position:50% 100%;}
h1 {font-family:Georgia, Times, serif; font-size:20px;}
h2 {text-align:center; font-family:Georgia, Times, serif; font-size:20px; width:100%; border-bottom:solid #999 1px;padding

In [79]:
from bs4 import BeautifulSoup
# create request header
headers = {'User-Agent': "andrewscott2@usf.edu"}

# get all companies data
response = requests.get(
    "https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm",
    headers=headers
    )
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.find)



<bound method PageElement.get_text of <?xml version="1.0" ?><!--XBRL Document Created with the Workiva Platform--><!--Copyright 2023 Workiva--><!--r:c68c2fd8-345e-4faa-8610-8b173d5da094,g:843081e3-ad04-4f75-a78c-e87ea3423788,d:1cb1ba018cb1455aa66bd3f9ab0c5b1a--><html xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:aapl="http://www.apple.com/20230930" xmlns:country="http://xbrl.sec.gov/country/2023" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:srt="http://fasb.org/srt/2023" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmln